In [41]:
import pandas as pd
import numpy as np
from collections import Counter


In [42]:
data = pd.read_csv(r"./cicids_static_data.csv")

In [43]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ data description ^^^^^^^^^^^^^^^^^^^^^^^^^^^
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-30
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# 
# Numerical (59), Categorical (20) 
# dtypes: float64(24), int64(54), object(1) Label
# <class 'pandas.core.frame.DataFrame'>
# RangeIndex: 25191 entries, 0 to 25190
# Data columns (total 79 columns):
#
# 6 NaN (14  Flow Bytes/s                  25185 non-null  float64)
# Duplicate rows: 433
# drop - constant value "0":  Bwd PSH Flags,  Bwd URG Flags, Fwd Avg Bytes/Bulk,  Fwd Avg Packets/Bulk, 
# drop - constant value "0":  Fwd Avg Bulk Rate,  Bwd Avg Bytes/Bulk,  Bwd Avg Packets/Bulk, Bwd Avg Bulk Rate
#
# negative - Init_Win_bytes_forward,  Init_Win_bytes_backward,  Flow Duration



In [44]:
def drop_na(X):
    # check for NaN values 
    print("the number of NaN:", X.isna().sum().sum())
    if (X.isna().sum().sum()) > 0:
        # drop NaN values
        X = X.dropna()
    return X


def drop_infinity(X):
    # check for values approaching infinity
    X = X.replace([np.inf, -np.inf], np.nan)
    print("the number of infinity:", X.isna().sum().sum())
    return X

In [45]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ features extraction ^^^^^^^^^^^^^^^^^^^^^^^^^^^
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-30
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

print(data.shape)
#(25191, 79)

# Step 1: drop NaN
data = drop_na(data)
print("after drop na:", data.shape)
#the number of NaN: 6
#after drop na: (25185, 79)

# Step 2: drop infinity
data = drop_infinity(data)
data = drop_na(data)
print("after drop infinity:", data.shape)
#the number of infinity: 12
#the number of NaN: 12
#after drop infinity: (25179, 79)

# Step 3: drop duplicate()
print(Counter(pd.DataFrame(data).duplicated()))
# Counter({False: 24747, True: 432})
data = pd.DataFrame(data).drop_duplicates(keep='first')
data = data.reset_index(drop=True)
print("after drop duplicates:", data.shape)
# after drop duplicates: (24747, 79)

(25191, 79)
the number of NaN: 6
after drop na: (25185, 79)
the number of infinity: 12
the number of NaN: 12
after drop infinity: (25179, 79)
Counter({False: 24747, True: 432})
after drop duplicates: (24747, 79)


In [46]:
# negative -  Flow Duration, Flow Bytes/s,  Flow Packets/s,  Flow IAT Mean,  Flow IAT Max,  
# negative -  Flow IAT Min, Init_Win_bytes_forward, Init_Win_bytes_backward
# negative -  Flow Duration, Flow Bytes/s,  Flow Packets/s,  Flow IAT Mean,  Flow IAT Max,  
# negative -  Flow IAT Min, Init_Win_bytes_forward, Init_Win_bytes_backward

def del_negative(D,X):
    for i in range(0,len(X.index)):
        if X.loc[i] < 0:
            D.drop(index=i,axis=0,inplace=True,errors='ignore')
            D = D.reset_index(drop=True)
            print(i)
            return D
        else:
            return D

print("before drop negative rows:",data.shape)
data = del_negative(data, data[' Flow Duration'])
data = del_negative(data, data['Flow Bytes/s'])
data = del_negative(data, data[' Flow Packets/s'])
data = del_negative(data, data[' Flow IAT Mean'])
data = del_negative(data, data[' Flow IAT Max'])
data = del_negative(data, data[' Flow IAT Min'])
data = del_negative(data, data['Init_Win_bytes_forward'])
data = del_negative(data, data[' Init_Win_bytes_backward'])
print("after drop negative rows:",data.shape)

before drop negative rows: (24747, 79)
0
0
after drop negative rows: (24745, 79)


In [47]:
# Label
lab_dict = {'ATTACK': 1, 'BENIGN' : 0}
data['Label'] = data['Label'].apply(lambda x: lab_dict[x])
data_y = data['Label']

# --------------- output 1-----------
data.to_csv("./2-1-1 5388 data_aftcat_full.csv",index=0)

list_l = ['Label']
data.drop(list_l, axis=1, inplace=True)
data.to_csv("./2-1-1 5388 data_aftcat_full_no label.csv",index=0)

In [48]:
data

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,7496,43,1,1,0,6,0,0,0.000000,0.000000,...,0,40,0.00000,0.0000,0,0,0.000000e+00,0.000,0,0
1,46564,61,1,1,6,6,6,6,6.000000,0.000000,...,0,20,0.00000,0.0000,0,0,0.000000e+00,0.000,0,0
2,80,69,2,0,0,0,0,0,0.000000,0.000000,...,0,20,0.00000,0.0000,0,0,0.000000e+00,0.000,0,0
3,80,98403702,7,5,377,11595,377,0,53.857143,142.492606,...,1,32,986.00000,0.0000,986,986,9.840000e+07,0.000,98400000,98400000
4,53,30843,1,1,54,70,54,54,54.000000,0.000000,...,0,40,0.00000,0.0000,0,0,0.000000e+00,0.000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24740,50100,197,1,1,6,6,6,6,6.000000,0.000000,...,0,20,0.00000,0.0000,0,0,0.000000e+00,0.000,0,0
24741,3390,48,1,1,0,6,0,0,0.000000,0.000000,...,0,40,0.00000,0.0000,0,0,0.000000e+00,0.000,0,0
24742,123,117,2,2,96,96,48,48,48.000000,0.000000,...,1,32,0.00000,0.0000,0,0,0.000000e+00,0.000,0,0
24743,443,116260101,28,25,1809,16411,961,0,64.607143,202.023897,...,27,20,75753.33333,143185.4198,530428,34284,9.612588e+06,1389284.644,10000000,5201068


In [49]:
# Step 4: drop columns, all value=0
col_list = data.columns

In [50]:
# drop - constant value "0":  Bwd PSH Flags,  Bwd URG Flags, Fwd Avg Bytes/Bulk,  Fwd Avg Packets/Bulk, 
# drop - constant value "0":  Fwd Avg Bulk Rate,  Bwd Avg Bytes/Bulk,  Bwd Avg Packets/Bulk, Bwd Avg Bulk Rate
print(Counter(data[' Bwd PSH Flags']),
    Counter(data[' Bwd URG Flags']),
    Counter(data['Fwd Avg Bytes/Bulk']),
    Counter(data[' Fwd Avg Packets/Bulk']),
    Counter(data[' Fwd Avg Bulk Rate']),
    Counter(data[' Bwd Avg Bytes/Bulk']),
    Counter(data[' Bwd Avg Packets/Bulk']),
    Counter(data['Bwd Avg Bulk Rate']),
)

print("before drop 0 columns:",data.shape)
list = [' Bwd PSH Flags',' Bwd URG Flags','Fwd Avg Bytes/Bulk',' Fwd Avg Packets/Bulk',' Fwd Avg Bulk Rate',' Bwd Avg Bytes/Bulk',' Bwd Avg Packets/Bulk','Bwd Avg Bulk Rate']
data.drop(list, axis=1, inplace=True)
print("after drop 0 columns:",data.shape)

Counter({0: 24745}) Counter({0: 24745}) Counter({0: 24745}) Counter({0: 24745}) Counter({0: 24745}) Counter({0: 24745}) Counter({0: 24745}) Counter({0: 24745})
before drop 0 columns: (24745, 78)
after drop 0 columns: (24745, 70)


In [54]:
# --------------- output 2 -------------------------
data_aftcat = data

data_aftcat.to_csv("./2-1-1 5388 data_aftcat.csv",index=0)
data_y.to_csv("./2-1-1 5388 data_aftcat label.csv",index=0)

In [55]:
data_y

0        1
1        0
2        0
3        1
4        0
        ..
24740    0
24741    1
24742    0
24743    0
24744    0
Name: Label, Length: 24745, dtype: int64

In [56]:
data_aftcat

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,7496,43,1,1,0,6,0,0,0.000000,0.000000,...,0,40,0.00000,0.0000,0,0,0.000000e+00,0.000,0,0
1,46564,61,1,1,6,6,6,6,6.000000,0.000000,...,0,20,0.00000,0.0000,0,0,0.000000e+00,0.000,0,0
2,80,69,2,0,0,0,0,0,0.000000,0.000000,...,0,20,0.00000,0.0000,0,0,0.000000e+00,0.000,0,0
3,80,98403702,7,5,377,11595,377,0,53.857143,142.492606,...,1,32,986.00000,0.0000,986,986,9.840000e+07,0.000,98400000,98400000
4,53,30843,1,1,54,70,54,54,54.000000,0.000000,...,0,40,0.00000,0.0000,0,0,0.000000e+00,0.000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24740,50100,197,1,1,6,6,6,6,6.000000,0.000000,...,0,20,0.00000,0.0000,0,0,0.000000e+00,0.000,0,0
24741,3390,48,1,1,0,6,0,0,0.000000,0.000000,...,0,40,0.00000,0.0000,0,0,0.000000e+00,0.000,0,0
24742,123,117,2,2,96,96,48,48,48.000000,0.000000,...,1,32,0.00000,0.0000,0,0,0.000000e+00,0.000,0,0
24743,443,116260101,28,25,1809,16411,961,0,64.607143,202.023897,...,27,20,75753.33333,143185.4198,530428,34284,9.612588e+06,1389284.644,10000000,5201068
